In [4]:
## La version de python utilisée est 3.12.7

!pip install -r requirements.txt -q

In [5]:
import pandas as pd
import requests
import lxml as lxml
from bs4 import BeautifulSoup
import io as io
import math
import gzip
import shutil
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import json
from pandasgui import show
import numpy as np
from io import BytesIO
from folium.plugins import HeatMap
import nbconvert

from script import process_data
from script import geolocaliser
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

# 2. Création du fichier des transactions cotières

On télécharge la base Demande de valeurs foncières (DVF) qui référence, pour l'année 2023, l'ensemble des mutations à titre onéreux (en majeure partie géolocalisées) : https://files.data.gouv.fr/geo-dvf/latest/csv/2023/full.csv.gz

Les informations sont issues de la Base nationale des données patrimoniales, alimentées par le système d'information de la DGFip et couvrent la France métropolitaine à l'exception des départements du Bas-Rhin, du Haut-Rhin et de Moselle.

La base recense des actes (id_mutation), qui comportent un ou plusieurs mutations distinctes, repérées par le numéro de disposition (numero_disposition).

Les observations de la base, appelées "lignes de restitution", concernent les différents locaux d'une mutation (Appartement, Maison, Dépendance, Local Industriel), ventilées selon autant de natures de culture présentes dans l'immeuble.

In [6]:
# url = "https://static.data.gouv.fr/resources/demandes-de-valeurs-foncieres/20241008-071041/valeursfoncieres-2023.txt.zip"

url = "https://files.data.gouv.fr/geo-dvf/latest/csv/2023/full.csv.gz"
# Envoyer une requête HTTP pour obtenir le fichier CSV

downloaded_file = "full.csv.gz"
response = requests.get(url)

with open(downloaded_file, 'wb') as file:
    file.write(response.content)

# Décompresser le fichier
with gzip.open(downloaded_file, 'rb') as f_in:
    with open("full.csv", 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

# Charger le fichier CSV dans un DataFrame
df = pd.read_csv("full.csv",encoding="utf-8")

# Optionnel : supprimer le fichier compressé après décompression
os.remove("full.csv.gz")
os.remove("full.csv")

C:\Users\flori\AppData\Local\Temp\ipykernel_13580\190562132.py:18: DtypeWarning: Columns (8,10,12,14,17,18,20,22,26,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("full.csv",encoding="utf-8")


In [7]:
print(df.shape[0])
print(df['id_mutation'].unique().shape[0])

3799407
1356412


# Première analyse : afficher toutes les variables, leurs types et le pourcentage des transactions non renseignées

# Premiers filtres
1) Suppression de toutes les colonnes non utilisées
2) Suppression de toutes les transactions non renseignées
3) Reconstitution des adresses complètes

4) Retenir toutes les transactions qui correspondent à des ventes
5) Géolocaliser toutes les transactions qui ne le sont pas dans la base originelle
6) Renseigner les départements et régions

7) On ne garde que les biens particuliers (appartements et maisons)
8) Vérifier que la surface est renseignée
9) Filtre sur les seuls appartements
10) Exclure les biens dans les DROM

In [8]:
## Première opération : alléger le dataset

colonnes_a_supprimer = ['adresse_suffixe',
                        'code_nature_culture',
                        'ancien_code_commune',
                        'ancien_nom_commune',
                        'ancien_id_parcelle',
                        'numero_volume',
                        'code_nature_culture_speciale',
                        'nature_culture_speciale',
                        'lot1_numero',
                        'lot2_numero',
                        'lot3_numero',
                        'lot4_numero',
                        'lot5_numero',
                        'lot1_surface_carrez',
                        'lot2_surface_carrez',
                        'lot3_surface_carrez',
                        'lot4_surface_carrez',
                        'lot5_surface_carrez'
                        ]

df.drop(columns=colonnes_a_supprimer, inplace=True)

In [9]:
# # Convertir les colonnes spécifiques en chaînes de caractères
# colonnes_a_convertir = ['adresse_code_voie', 'code_commune', 'code_departement']
# df_concatene = process_data.convertir_en_str(df_concatene, colonnes_a_convertir)

In [10]:
# # URL de base pour les fichiers DVF
# base_url = "https://files.data.gouv.fr/geo-dvf/latest/csv/{year}/full.csv.gz"

# # Liste des années
# annees = range(2019, 2024)

# # Colonnes à supprimer
# colonnes_a_supprimer = [
#     'id_mutation', 'adresse_suffixe', 'numero_disposition',
#     'code_nature_culture', 'ancien_code_commune', 'ancien_nom_commune',
#     'ancien_id_parcelle', 'numero_volume', 'code_nature_culture_speciale',
#     'nature_culture_speciale', 'lot1_numero', 'lot2_numero', 'lot3_numero',
#     'lot4_numero', 'lot5_numero', 'lot1_surface_carrez', 'lot2_surface_carrez',
#     'lot3_surface_carrez', 'lot4_surface_carrez', 'lot5_surface_carrez'
# ]

On importe le fichier des communes cotières et on ne retient que les transactions liées à ces communes

In [11]:
# Effectuer un left join entre df_final et df_communes
df_cotieres = pd.read_csv('data/communes_cotieres.csv',sep=";")
df_cotieres = df_cotieres.rename(columns={'latitude': 'latitude_centre'})
df_cotieres = df_cotieres.rename(columns={'longitude': 'longitude_centre'})

liste_cotieres = df_cotieres['nom'].unique().tolist()

In [12]:
df = df[df['nom_commune'].isin(liste_cotieres)]

In [13]:
print(df.shape[0])
print(df['id_mutation'].unique().shape[0])

286003
123784


In [14]:
#Filtre 1 : On supprime tous les montants non renseignés
print(df['valeur_fonciere'].isna().sum())
df = df.dropna(subset=['valeur_fonciere'])
df = df[df['valeur_fonciere']>0]

1290


In [15]:
# Filtre 2 : On ne conserve que les ventes (hors vente du neuf)
# df = df[df['nature_mutation']=='Vente']
df = df[df['nature_mutation'].str.contains('Vente', na=False)]
df = df.drop(columns=['nature_mutation'])

In [16]:
# 3ème opération : conversion des adresses en string

print(df['adresse_numero'].dtype)
print(df['adresse_nom_voie'].dtype)
print(df['code_postal'].dtype)
print(df['nom_commune'].dtype)

colonnes_a_nettoyer = ['adresse_numero', 'code_postal']
df = process_data.nettoyer_colonnes(df, colonnes_a_nettoyer)
# Convertir la colonne 'code_commune' en type string
df['code_commune'] = df['code_commune'].astype('string')

# Ajouter un '0' au début si la chaîne a 4 caractères
df['code_commune'] = [x.zfill(5) if len(x) == 4 else x for x in df['code_commune']]

# Vérifier les résultats
print(df['code_commune'].head())

float64
object
float64
object
62753     02316
73247     02316
73248     02316
75464     02316
135339    06159
Name: code_commune, dtype: object


In [17]:
# 4ème opération (si besoin de géolocalisation) : Compléter par le type de voie

voie = pd.read_csv("data/voie.csv",sep=";",encoding="utf-8")
print(voie.head())

# Liste des abréviations de types de voie
abbreviations = voie['abreviation'].tolist()

# Appliquer la fonction à la colonne 'adresse_nom_voie'
result = [process_data.check_abbreviation(adresse,abbreviations) for adresse in df['adresse_nom_voie']]

# Décomposer les résultats dans les colonnes 'type_voie' et 'nom_voie'
df['type_voie'] = [x[0] for x in result]
df['nom_voie'] = [x[1] for x in result]

df = df.merge(voie,left_on=['type_voie'],right_on=['abreviation'])

  abreviation type_voie_complet
0         RUE               Rue
1          AV            Avenue
2         RTE             Route
3         CHE            Chemin
4          BD         Boulevard


In [18]:
## 5ème : Réécriture de l'adresse

df['Adresse'] = df['adresse_numero'] + ' ' + df['type_voie_complet'] + ' ' + df['nom_voie'] + ' ' + df['code_postal'] + ' ' + df['nom_commune']

print(df['Adresse'].head())

0                       9 Rue DE LA PLAINE 2600 Fleury
1                       9 Rue DE LA PLAINE 2600 Fleury
2     6 Avenue MARECHAL FOCH 6230 Villefranche-sur-Mer
3    17 Avenue FRANCOIS DE MONLEON 6190 Roquebrune-...
4    17 Avenue FRANCOIS DE MONLEON 6190 Roquebrune-...
Name: Adresse, dtype: string


In [19]:
# Charger les données des codes de région
region = pd.read_csv('data/code_region.csv', sep=';')
print(region['departmentCode'].dtype)
print(df['code_departement'].dtype)

# Afficher les premières lignes pour vérifier
print(region.head())

# Vérifier le type de la colonne 'departmentCode'
print(f"Type de la colonne 'departmentCode': {region['departmentCode'].dtype}")

# Vérifier les modalités (valeurs uniques) de 'departmentCode'
print("Modalités de 'departmentCode':")
print(region['departmentCode'].unique())
print(df['code_departement'].unique())

object
object
  departmentCode           departmentName  regionCode  \
0              1                      Ain        84.0   
1              2                    Aisne        32.0   
2              3                   Allier        84.0   
3              4  Alpes-de-Haute-Provence        93.0   
4              5             Hautes-Alpes        93.0   

                   regionName  
0        Auvergne-Rhône-Alpes  
1             Hauts-de-France  
2        Auvergne-Rhône-Alpes  
3  Provence-Alpes-Côte d'Azur  
4  Provence-Alpes-Côte d'Azur  
Type de la colonne 'departmentCode': object
Modalités de 'departmentCode':
['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15' '16'
 '17' '18' '19' '21' '22' '23' '24' '25' '26' '27' '28' '29' '2A' '2B'
 '30' '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41' '42' '43'
 '44' '45' '46' '47' '48' '49' '50' '51' '52' '53' '54' '55' '56' '57'
 '58' '59' '60' '61' '62' '63' '64' '65' '66' '67' '68' '69' '70' '71'
 '72' '73' '74' '75'

In [20]:
region['departmentCode'] = region['departmentCode'].astype(str)
df['code_departement'] = df['code_departement'].astype(str)

print(region['departmentCode'].unique())
print(df['code_departement'].unique())

['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15' '16'
 '17' '18' '19' '21' '22' '23' '24' '25' '26' '27' '28' '29' '2A' '2B'
 '30' '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41' '42' '43'
 '44' '45' '46' '47' '48' '49' '50' '51' '52' '53' '54' '55' '56' '57'
 '58' '59' '60' '61' '62' '63' '64' '65' '66' '67' '68' '69' '70' '71'
 '72' '73' '74' '75' '76' '77' '78' '79' '80' '81' '82' '83' '84' '85'
 '86' '87' '88' '89' '90' '91' '92' '93' '94' '95' '971' '972' '973' '974'
 '976' '987' '988']
['2' '6' '8' '11' '13' '14' '17' '19' '21' '22' '24' '25' '27' '28' '29'
 '2A' '2B' '30' '32' '33' '34' '35' '38' '40' '42' '44' '45' '50' '51'
 '52' '54' '56' '58' '59' '60' '62' '64' '65' '66' '69' '76' '77' '80'
 '83' '85' '88' '89' '972' '974']


In [21]:
#Fusionner df_geolocalisees avec les codes de région
df = df.merge(region, left_on=["code_departement"], right_on=['departmentCode'], how='left')

df['regionCode'] = df['regionCode'].astype('Int64')  # Utilise Int64 pour gérer les valeurs manquantes
#Compter le nombre de transactions par région

transactions_par_region = df.groupby('regionName').size()

# Calculer le total des transactions
total_transactions = transactions_par_region.sum()

# Calculer la part de chaque région dans le total des transactions
part_region = (transactions_par_region / total_transactions) * 100

# Créer un DataFrame avec les résultats
tableau_regions = pd.DataFrame({
    'Nombre de transactions': transactions_par_region,
    'Part du total (%)': part_region
})

# Afficher le tableau des régions
print(tableau_regions)

                            Nombre de transactions  Part du total (%)
regionName                                                           
Auvergne-Rhône-Alpes                            99           0.043931
Bourgogne-Franche-Comté                         68           0.030175
Bretagne                                     37557          16.665853
Centre-Val de Loire                             24           0.010650
Corse                                         4507           1.999973
Grand Est                                      119           0.052806
Hauts-de-France                              14516           6.441450
La Réunion                                    1050           0.465936
Martinique                                      50           0.022187
Normandie                                    18787           8.336698
Nouvelle-Aquitaine                           24238          10.755570
Occitanie                                    19522           8.662853
Pays de la Loire    

In [22]:
print(df['type_local'].value_counts())

type_local
Dépendance                                  96656
Appartement                                 59213
Maison                                      34699
Local industriel. commercial ou assimilé     8423
Name: count, dtype: int64


3. Calcul de la base des prix au mètre carré

Notre objectif est d'évaluer le prix au mètre carré des biens en distinguant les maisons et les appartements
Une mutation peut contenir plusieurs types de locaux

On conserve chaque couple mutation/disposition qui contient une nature de culture sols ou vide. On suppose que ces mutations concernent des biens à visée d'habitation. Pour ces mutations, on regarde si elles contiennent :
- un local de type 'Maison' :  la transaction est référencée comme une maison
- un local de type 'Appartement' (mais sans 'Maison') : la transaction est référencée comme un appartement
- aucun local de type 'Maison' ou 'Appartement' : la transaction n'est pas retenue

La valeur foncière étant dupliquée, seule sa première occurence est retenue. Le nombre de pièces retenu est le plus grand de toutes les lignes de restitution. Les surfaces réelles sont additionnées par mutation.

In [23]:
# Fonction pour traiter chaque groupe
def process_group(group):
    # Vérifier si 'Maison' est présent dans type_local
    maison_present = 'Maison' in group['type_local'].values
    appart_present = 'Appartement' in group['type_local'].values and 'Maison' not in group['type_local'].values
    
    # Vérifier si 'nature_culture' contient autre chose que 'sols' ou est vide
    nature_culture_sols_ou_vide = group['nature_culture'].isna().any() or group['nature_culture'].str.contains('sols', na=False).any()
    
    # Remplacer NaN par 0 pour la somme des surfaces et de la valeur foncière
    surface_reelle_bati = group['surface_reelle_bati'].fillna(0).sum()
    surface_terrain = group['surface_terrain'].fillna(0).sum()
    valeur_fonciere = group['valeur_fonciere'].iloc[0]
    nombre_pieces_principales = group['valeur_fonciere'].max()
    
    # Rassembler les résultats
    result = {
        'valeur_fonciere': valeur_fonciere,
        'surface_reelle_bati': surface_reelle_bati,
        'surface_terrain': surface_terrain,
        'nombre_locaux': len(group),
        'maison_present': maison_present,
        'appart_present': appart_present,
        'nature_culture_sols_ou_vide': nature_culture_sols_ou_vide,
        'nombre_pieces_principales': nombre_pieces_principales
    }
    
    # Conserver toutes les autres colonnes (première valeur par défaut)
    for col in group.columns:
        if col not in result:
            result[col] = group[col].iloc[0]
    
    return pd.Series(result)

# Appliquer le traitement
df_mai = df.groupby('id_mutation').apply(process_group)

# Utiliser `reset_index()` sans insérer `id_mutation` à nouveau
df_mai.reset_index(drop=True, inplace=True)

# Vérifier le résultat
print(df_mai.head())


   valeur_fonciere  surface_reelle_bati  surface_terrain  nombre_locaux  \
0          35000.0                  8.0              0.0              1   
1         175000.0                 54.0           1401.0              3   
2          85000.0                 38.0              0.0              2   
3         194000.0                 52.0             15.0              5   
4          96000.0                 55.0              0.0              1   

   maison_present  appart_present  nature_culture_sols_ou_vide  \
0           False           False                         True   
1            True           False                         True   
2           False            True                         True   
3           False            True                         True   
4           False            True                         True   

   nombre_pieces_principales   id_mutation date_mutation  ...   latitude  \
0                    35000.0  2023-1000010    2023-06-08  ...  49.920737   


C:\Users\flori\AppData\Local\Temp\ipykernel_13580\4046070118.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_mai = df.groupby('id_mutation').apply(process_group)


In [24]:
df = df_mai[df_mai['nature_culture_sols_ou_vide']==True]

In [25]:
df = df[(df['appart_present'] == True) | (df['maison_present'] == True)]

In [26]:
def assign_type_local(row):
    if row['maison_present']:
        return 'Maison'
    elif row['appart_present']:
        return 'Appartement'
    else:
        return np.nan

df['type_local'] = df.apply(assign_type_local, axis=1)
df = df.drop(columns=['appart_present','maison_present','nature_culture'])

On vérifie que toutes les transactions concernent des surfaces non nulles

In [27]:
df = df[df['surface_reelle_bati'] > 0]

In [28]:
df['prix_m2'] = df['valeur_fonciere'] / df['surface_reelle_bati']

In [29]:
communes_riches = df.groupby('nom_commune')

In [30]:
df = pd.merge(df, df_cotieres, how='left', left_on='nom_commune', right_on='nom')

In [31]:
# One ne garde donc que les maisons et les appartements

df = df[df['type_local'].isin(['Maison', 'Appartement'])]

total_transactions = df.shape[0]
total_transactions2 = df.shape[0]

# Calcul du nombre d'appartements et de maisons dans ce sous-ensemble
appartements = df[df['type_local'] == 'Appartement'].shape[0]
maisons = df[df['type_local'] == 'Maison'].shape[0]

# Calcul des pourcentages
pourcentage = total_transactions2/total_transactions * 100
pourcentage_appartements = (appartements / total_transactions2) * 100
pourcentage_maisons = (maisons / total_transactions2) * 100

# Générer la phrase
phrase = f"Sur les {total_transactions} transactions renseignées et géolocalisées, on ne conserve que {total_transactions2} (soit {pourcentage:.1f}%)" \
         f"transactions représentant {pourcentage_appartements:.1f}% d'appartements et {pourcentage_maisons:.1f}% de maisons."

# Afficher la phrase
print(phrase)

Sur les 92156 transactions renseignées et géolocalisées, on ne conserve que 92156 (soit 100.0%)transactions représentant 62.0% d'appartements et 38.0% de maisons.


In [32]:
# Une partie des transactions sont en doublon

# Compter les doublons en fonction des colonnes spécifiées
nb_doublons = df.duplicated(subset=['id_mutation']).sum()
print(f"Nombre de doublons : {nb_doublons}")

# Supprimer les doublons en gardant la première occurrence
df = df.drop_duplicates(subset=['adresse_nom_voie', 'code_postal', 'surface_reelle_bati', 'valeur_fonciere'], keep='first')

# Vérifier le nouveau nombre de lignes
print(f"Nombre de lignes après suppression des doublons : {len(df)}")


Nombre de doublons : 11695
Nombre de lignes après suppression des doublons : 80152


In [33]:
# Calcul de la moyenne des prix totale en fonction de la surface
prix_moyen_m2 = df['valeur_fonciere'].sum() / df['surface_reelle_bati'].sum()
print(f"Moyenne totale des prix en fonction de la surface : {prix_moyen_m2:.2f} €")

# Groupement par région et type_local pour calculer les prix moyens
prix_moyen = (
    df.groupby(['regionName', 'type_local'])
    .apply(lambda group: group['valeur_fonciere'].sum() / group['surface_reelle_bati'].sum())
    .reset_index(name='prix_moyen')
)

# Conversion en tableau croisé dynamique (pivot table)
tableau_prix_moyen = prix_moyen.pivot_table(
    index='regionName',
    columns='type_local',
    values='prix_moyen',
    aggfunc='mean'  # Non nécessaire ici car chaque cellule est déjà agrégée
)

# Remplir les valeurs manquantes par 0 ou autre
tableau_prix_moyen = tableau_prix_moyen.fillna(0)

# Calcul du nombre de transactions par région
nb_transactions_region = df.groupby('regionName').size()

# Calcul du pourcentage d'appartements et de maisons par région
nb_transactions_appart = df[df['type_local'] == "Appartement"].groupby('regionName').size()
nb_transactions_maison = df[df['type_local'] == "Maison"].groupby('regionName').size()

# Calculer le pourcentage pour chaque type par région
pourcentage_appart = (nb_transactions_appart / nb_transactions_region * 100).fillna(0)
pourcentage_maison = (nb_transactions_maison / nb_transactions_region * 100).fillna(0)


Moyenne totale des prix en fonction de la surface : 3452.00 €

C:\Users\flori\AppData\Local\Temp\ipykernel_13580\1610620510.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: group['valeur_fonciere'].sum() / group['surface_reelle_bati'].sum())


In [34]:
# URL du fichier GeoJSON des régions de France
url_region = "https://france-geojson.gregoiredavid.fr/repo/regions.geojson"

# Récupérer le fichier GeoJSON des régions de France
response = requests.get(url_region)
regions_geojson = response.json()

# Créer une liste pour stocker les noms des régions
region_names = []

# Extraire les noms des régions depuis les propriétés de chaque feature
for feature in regions_geojson['features']:
    region_name = feature['properties']['nom']
    region_names.append(region_name)

# Créer un DataFrame avec les noms des régions
df_regions = pd.DataFrame(region_names, columns=['regionName'])
    
# Afficher le DataFrame
print(df_regions)

 # Afficher les modalités de la colonne 'regionName'
modalites_region = df['regionName'].value_counts().reset_index()

modalites_region = pd.DataFrame(modalites_region[['regionName']])
# Affichage
print(modalites_region)


                    regionName
0                Île-de-France
1          Centre-Val de Loire
2      Bourgogne-Franche-Comté
3                    Normandie
4              Hauts-de-France
5                    Grand Est
6             Pays de la Loire
7                     Bretagne
8           Nouvelle-Aquitaine
9                   Guadeloupe
10                  Martinique
11                      Guyane
12                  La Réunion
13                     Mayotte
14                   Occitanie
15        Auvergne-Rhône-Alpes
16  Provence-Alpes-Côte d'Azur
17                       Corse
                    regionName
0   Provence-Alpes-Côte d'Azur
1                     Bretagne
2           Nouvelle-Aquitaine
3                    Occitanie
4                    Normandie
5              Hauts-de-France
6             Pays de la Loire
7                        Corse
8                   La Réunion
9         Auvergne-Rhône-Alpes
10                   Grand Est
11               Île-de-France
12     B

## Travail sur les zones cotières
1) Import des listes des communes littorales
2) Calcul des centroïdes des communes (afin d'orienter la visualisation sur la commune)


In [35]:
# Création de la variable adresse_py en remplaçant les espaces par des "+"
df['adresse_py'] = df['Adresse'].str.replace(' ', '+', regex=True)

In [36]:
commune = pd.read_csv('data/communes_code.csv', sep=';',encoding="utf-8")
show(commune)
# Left_join des codes_communes avec les noms des communes geoJson
df = df.merge(commune, left_on=["code_commune"], right_on=['code'], how='left')
df = df.sort_values(by='code_commune', ascending=True)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
C:\Users\flori\AppData\Local\Temp\ipykernel_13580\1226572168.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  show(commune)
C:\Users\flori\AppData\Local\Temp\ipykernel_13580\1226572168.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  show(commune)
C:\Users\flori\AppData\Local\Temp\ipykernel_13580\1226572168.py:2: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\App

In [37]:
# Combien de transactions ne sont pas géolocalisées ?

# Nombre total de transactions renseignées (celles qui ont des valeurs dans 'latitude' et 'longitude')
total_transactions_renseignees = df.shape[0]

# Nombre de lignes où 'latitude' ou 'longitude' est manquant (sans double compte)
non_geolocalisees = df[df['latitude'].isna() | df['longitude'].isna()].shape[0]

# Calcul du pourcentage de transactions non géolocalisées
pourcentage_non_geolocalisees = (non_geolocalisees / total_transactions_renseignees) * 100

# Affichage de la phrase
print(f"Sur les {total_transactions_renseignees} transactions renseignées, {pourcentage_non_geolocalisees:.2f}% ne sont pas géolocalisées.")

Sur les 80152 transactions renseignées, 0.41% ne sont pas géolocalisées.


In [38]:
# Calculer le nombre de transactions par commune
nombre_transactions_cotieres = df.groupby('communeName').size().reset_index(name='Nombre de transactions')

show(nombre_transactions_cotieres)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI


C:\Users\flori\AppData\Local\Temp\ipykernel_13580\2325933965.py:4: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel_13580\2325933965.py:4: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel_13580\2325933965.py:4: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel_13580\2325933965.py:4: FutureWarning:

Series.__getit

In [39]:
df_missing = df[df['latitude'].isna()]

In [40]:
df_missing = df_missing.drop(columns=['latitude', 'longitude'])

In [41]:
df_missing = geolocaliser.geolocaliser_actifs(df_missing, 'adresse_py', 'latitude', 'longitude')

In [42]:
# Concatène les DataFrames en réinitialisant l'index
df_final = pd.concat([df, df_missing], ignore_index=True)

In [43]:
base_transactions = df_final.drop(columns=['departmentCode','communeName','code'])

In [44]:
base_transactions.to_csv("data/base.csv",sep=";",index=False,encoding="utf-8")

In [45]:
test = base_transactions[base_transactions['type_local']=='Maison']

In [46]:
show(test)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
C:\Users\flori\AppData\Local\Temp\ipykernel_13580\3307407764.py:1: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel_13580\3307407764.py:1: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel_13580\3307407764.py:1: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel_13